In [25]:
import pyterrier as pt
import pickle
import pandas as pd
import numpy as np

In [2]:
df = pd.read_json("./hiphop.json")

In [3]:
df['docno'] = df['id']
df['text'] = df['title']

### Index mit mehreren Feldern anlegen

In [4]:
df_dict = df.to_dict(orient='records')

In [7]:
index_folder = "./index_hiphop"

fields=['text', 'tags', 'authors', 'abstract']

indexer = pt.IterDictIndexer(index_folder, meta={'docno': 420, 'text': 4200}, overwrite=True)
index_ref = indexer.index(df_dict, fields=fields)

print(f"Index-Pfad: {index_ref.toString()}")

Index-Pfad: ./index_hiphop/data.properties


In [8]:
pickle.dump(df, open("./hiphop_search.pkl", "wb"))

In [9]:
index = pt.IndexFactory.of(index_ref)

## Statistiken, etc.

In [10]:
print(index.getCollectionStatistics().toString())

Number of documents: 1934
Number of terms: 11594
Number of postings: 72031
Number of fields: 4
Number of tokens: 101043
Field names: [text, tags, authors, abstract]
Positions:   false



In [11]:
df.columns

Index(['title', 'authors', 'year', 'isbn', 'publisher', 'abstract', 'tags',
       'url', 'id', 'type', 'search term', 'docno', 'text'],
      dtype='object')

In [12]:
for kv in index.getLexicon():
    print(f"{kv.getKey()} -> {kv.getValue().toString()}")

0 -> term1687 Nt=112 TF=273 maxTF=11 @{0 0 0} TFf=6,1,0,266
000 -> term3240 Nt=2 TF=2 maxTF=1 @{0 206 0} TFf=0,0,0,2
0001 -> term7528 Nt=2 TF=3 maxTF=2 @{0 212 0} TFf=0,0,0,3
001 -> term3729 Nt=12 TF=15 maxTF=3 @{0 217 6} TFf=0,0,0,15
0017 -> term1666 Nt=1 TF=1 maxTF=1 @{0 244 2} TFf=0,0,0,1
002 -> term8430 Nt=2 TF=3 maxTF=2 @{0 246 5} TFf=0,0,0,3
003 -> term10840 Nt=1 TF=1 maxTF=1 @{0 252 3} TFf=0,0,0,1
005 -> term8871 Nt=1 TF=1 maxTF=1 @{0 255 6} TFf=0,0,0,1
006 -> term9967 Nt=2 TF=2 maxTF=1 @{0 259 1} TFf=0,0,0,2
008 -> term7778 Nt=1 TF=1 maxTF=1 @{0 265 3} TFf=0,0,0,1
009 -> term8784 Nt=2 TF=2 maxTF=1 @{0 268 6} TFf=0,0,0,2
01 -> term7056 Nt=10 TF=13 maxTF=3 @{0 275 2} TFf=0,0,0,13
013 -> term11455 Nt=1 TF=2 maxTF=2 @{0 297 4} TFf=0,0,0,2
018 -> term11507 Nt=1 TF=1 maxTF=1 @{0 301 1} TFf=0,0,0,1
01a35 -> term7194 Nt=1 TF=1 maxTF=1 @{0 304 4} TFf=0,1,0,0
02 -> term7532 Nt=2 TF=2 maxTF=1 @{0 307 7} TFf=0,0,0,2
021 -> term11497 Nt=1 TF=1 maxTF=1 @{0 313 5} TFf=0,0,0,1
022 -> term8762 

In [13]:
term = "polit"
index.getLexicon()[term].toString()

'term885 Nt=11 TF=14 maxTF=2 @{0 103378 6} TFf=4,1,0,9'

In [14]:
term_freq_dict = {}

for kv in index.getLexicon():
    term_freq_dict[kv.getKey()] = kv.getValue().frequency

In [15]:
term_freq_dict_sorted = sorted(term_freq_dict.items(), key=lambda x: x[1], reverse=True)

In [16]:
term_freq_dict_sorted[:42]

[('hip', 1492),
 ('null', 1015),
 ('theori', 823),
 ('bishop', 687),
 ('dblp', 670),
 ('lattic', 669),
 ('2', 554),
 ('cerebr', 549),
 ('palsi', 540),
 ('model', 536),
 ('studi', 473),
 ('patient', 456),
 ('result', 455),
 ('1', 411),
 ('disloc', 381),
 ('black', 350),
 ('3', 333),
 ('can', 330),
 ('effect', 328),
 ('gener', 324),
 ('field', 318),
 ('us', 308),
 ('function', 307),
 ('gaug', 303),
 ('oper', 294),
 ('group', 285),
 ('4', 275),
 ('0', 273),
 ('method', 273),
 ('mass', 272),
 ('siphon', 267),
 ('base', 255),
 ('control', 255),
 ('child', 253),
 ('fermion', 251),
 ('hole', 249),
 ('children', 247),
 ('string', 247),
 ('symmetri', 247),
 ('qcd', 245),
 ('system', 243),
 ('show', 239)]

## TF-IDF-Suche

In [17]:
search_engine_tfidf = pt.terrier.Retriever(index, wmodel="TF_IDF")

In [18]:
search_engine_tfidf.search("politics rap")

,qid,docid,docno,rank,score,query
0,1,82,https://www.bibsonomy.org/bibtex/20724ec534ea5...,0,10.546432,politics rap
1,1,946,https://www.bibsonomy.org/bibtex/2f34832f94c0d...,1,6.491343,politics rap
2,1,231,https://www.bibsonomy.org/bibtex/2ee0637e90c52...,2,6.284521,politics rap
3,1,21,https://www.bibsonomy.org/bibtex/2d69a29569dd9...,3,6.020302,politics rap
4,1,489,https://www.bibsonomy.org/bibtex/2684f23c73b5f...,4,5.946275,politics rap
5,1,150,https://www.bibsonomy.org/bibtex/20092aa05be5e...,5,5.879060,politics rap
6,1,157,https://www.bibsonomy.org/bibtex/2e8b5c548a89c...,6,5.792576,politics rap
7,1,3,https://www.bibsonomy.org/bibtex/28b47b4ba82ff...,7,5.305340,politics rap
8,1,76,https://www.bibsonomy.org/bibtex/2abae0a38f92b...,8,4.818443,politics rap
9,1,383,https://www.bibsonomy.org/bibtex/2b05d6d9be939...,9,4.633032,politics rap


### P und R

P = 0,57

R = 8/REL

In [19]:
8/14

0.5714285714285714

## TF-Suche

In [20]:
search_engine_tf = pt.BatchRetrieve(index, wmodel="Tf")

/tmp/ipykernel_4037/3266775375.py:1: DeprecationWarning: Call to deprecated class BatchRetrieve. (use pt.terrier.Retriever() instead) -- Deprecated since version 0.11.0.
  search_engine_tf = pt.BatchRetrieve(index, wmodel="Tf")


In [21]:
search_engine_tf.search("politics rap")

,qid,docid,docno,rank,score,query
0,1,82,https://www.bibsonomy.org/bibtex/20724ec534ea5...,0,4.0,politics rap
1,1,76,https://www.bibsonomy.org/bibtex/2abae0a38f92b...,1,2.0,politics rap
2,1,157,https://www.bibsonomy.org/bibtex/2e8b5c548a89c...,2,2.0,politics rap
3,1,231,https://www.bibsonomy.org/bibtex/2ee0637e90c52...,3,2.0,politics rap
4,1,383,https://www.bibsonomy.org/bibtex/2b05d6d9be939...,4,2.0,politics rap
5,1,889,https://www.bibsonomy.org/bibtex/2c54073464108...,5,2.0,politics rap
6,1,3,https://www.bibsonomy.org/bibtex/28b47b4ba82ff...,6,1.0,politics rap
7,1,21,https://www.bibsonomy.org/bibtex/2d69a29569dd9...,7,1.0,politics rap
8,1,33,https://www.bibsonomy.org/bibtex/20942846894ed...,8,1.0,politics rap
9,1,64,https://www.bibsonomy.org/bibtex/222f6540212cf...,9,1.0,politics rap


In [22]:
search_engine_tf.search("politics rap").docno[12]

'https://www.bibsonomy.org/bibtex/2684f23c73b5f1005d2a28744e3f1d8a9/whiteness'

### Berechnung IDF zum Prüfen der TF-IDF-Berechnung

In [28]:
from nltk.stem import PorterStemmer
stemmer = PorterStemmer()

In [29]:
def get_idf_for_term(term, index):
    lex = index.getLexicon()
    stemmed_term = stemmer.stem(term)

    if not stemmed_term in lex:
        return

    lex_entry = lex[stemmed_term]

    df_term = lex_entry.getDocumentFrequency()
    N = index.getCollectionStatistics().numberOfDocuments

    # inverse document frequency
    idf = N/df_term

    # apply logarithm(base 10) to idf
    log_idf = np.emath.logn(10, idf)

    return log_idf

In [30]:
term = "politics"
get_idf_for_term(term, index)

2.2450637845887575

## Code um TF-IDF selbst zu berechnen

#### funktioniert, im Vergleich zum selbst ausgerechneten TF-IDF, aber anderer Wert als search_engine_tfidf 

Beispiel: docid 82: 

score-TF: 4 -> rap=3, politics = 1

idf rap = 2.5083052193633386

idf politics = 2.2450637845887575

3 * 2.5083052193633386 + 1 * 2.2450637845887575 = 9.769979442678775

calc_tf_idf: 9.769979442678775

search_engine_tfidf: 10.612571

erster Tipp für den Unterschied wäre Rundungsdifferenzen, aber bei in der Mitte gerankten Ergebnissen ist die Differenz größer.

Bsp: docid 489
calc_: 2.2450637845887575
search-engine: 5.677485

#### Dieser Eintrag  ist als Ergebnis auch nicht relevant...


In [83]:
def calc_tf_idf(query, docno, index):

    lex = index.getLexicon()
    terms = query.split()
    tf_scores = []

    for term in terms:
        
        stemmed_term = stemmer.stem(term)
    
        if not stemmed_term in lex:
            
            continue
        
        lex_entry = lex[stemmed_term]
    
        df_term = lex_entry.getDocumentFrequency()
        N = index.getCollectionStatistics().numberOfDocuments
    
        idf = N/df_term    
        log_idf = np.emath.logn(10, idf)

        tf_term = search_engine_tf.search(term)[search_engine_tf.search(term)["docno"] == docno]["score"]

        if tf_term.empty:
            continue
            
        tf_int = int(tf_term)
        
        tf_idf = tf_int*log_idf

        tf_scores.append(tf_idf)
        
    return sum(tf_scores)

In [102]:
query = "politics rap"
docno = "https://www.bibsonomy.org/bibtex/2684f23c73b5f1005d2a28744e3f1d8a9/whiteness"

calc_tf_idf(query, docno, index)

/tmp/ipykernel_9800/3554385174.py:28: FutureWarning: Calling int on a single element Series is deprecated and will raise a TypeError in the future. Use int(ser.iloc[0]) instead
  tf_int = int(tf_term)


2.2450637845887575